In [26]:
import numpy as np
import pandas as pd
import bokeh
from bokeh.models import Circle, ColumnDataSource, ColorBar,HoverTool,\
Select, CustomJS,CustomJSFilter,CDSView,IndexFilter,Title
from bokeh.plotting import figure, show, output_notebook,output_file
from bokeh.tile_providers import Vendors, get_provider
from bokeh.palettes import brewer, all_palettes, Spectral6, Inferno
from bokeh.transform import linear_cmap
from bokeh.layouts import column, gridplot
from bokeh.io import output_file, show
from bokeh.models import Dropdown, Panel
from bokeh.layouts import column, row, WidgetBox, layout

In [2]:
def duration_distribution_plot(duration_data):
    '''
     Build a box plot for the distribution of the planned and actual duration values per input cluster
    :param planned_duration_vals: Planned duration values for cluster activities
    :param actual_duration_vals: Actual duration values for cluster activities
    '''
    #source = ColumnDataSource(data)
    #filter = IndexFilter(indices=list(data.index))
    #view = CDSView(source=source, filters=[filter])
    #plot = figure(x_axis_type="mercator", y_axis_type="mercator",plot_height=700, plot_width=700)
    #plot.title.text_color = 'purple'
    # tooltips=[('Practice', '@practice'), ('CCG', '@ccg'), (first_value.capitalize(),hover_first_value)]
    pass

In [12]:
duration_ratios = np.array([4.8, 3.95, 0.73, 3.27, 17.2])
planned_duration_vals = np.array([5, 21, 15, 26, 5])
actual_duration_vals =  np.array([24, 83, 11, 85, 86])

In [22]:
duration_type = len(planned_duration_vals) * ['Planned'] + len(actual_duration_vals) *['Actual']
duration_vals = list(planned_duration_vals)+list(actual_duration_vals)
duration_df = pd.DataFrame(list(zip(duration_type, duration_vals)), columns=['type', 'value'])
duration_df

,type,value
0,Planned,5
1,Planned,21
2,Planned,15
3,Planned,26
4,Planned,5
5,Actual,24
6,Actual,83
7,Actual,11
8,Actual,85
9,Actual,86


In [31]:
import bokeh.plotting
# Use Bokeh chart to make plot
p = bokeh.plotting.BoxPlot(duration_df, values='value', label='type',
                        color='type', ylabel='Days')
# Display it
bokeh.io.show(p)

AttributeError: module 'bokeh.plotting' has no attribute 'BoxPlot'

In [34]:
''' A `Box Plot`_ of synthetic data. This example demonstrates combining
multiple basic glyphs to create a more complicated chart.
.. bokeh-example-metadata::
    :apis: bokeh.plotting.figure.vbar, bokeh.plotting.figure.rect, bokeh.plotting.figure.segment
    :refs: :ref:`userguide_plotting` > :ref:`userguide_plotting_segments_rays`, :ref:`userguide_plotting_bars_rects`
    :keywords: bars, boxplot, categorical, pandas
.. _Box Plot: https://en.wikipedia.org/wiki/Box_plot
'''
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show

# generate some synthetic time series for six different categories
cats = list("abcdef")
yy = np.random.randn(2000)
g = np.random.choice(cats, 2000)
for i, l in enumerate(cats):
    yy[g == l] += i // 2
df = pd.DataFrame(dict(score=yy, group=g))

# find the quartiles and IQR for each category
groups = df.groupby('group')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group.score > upper.loc[cat]['score']) | (group.score < lower.loc[cat]['score'])]['score']
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = list(out.index.get_level_values(0))
    outy = list(out.values)

p = figure(tools="", background_fill_color="#efefef", x_range=cats, toolbar_location=None)

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper.score = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'score']),upper.score)]
lower.score = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'score']),lower.score)]

# stems
p.segment(cats, upper.score, cats, q3.score, line_color="black")
p.segment(cats, lower.score, cats, q1.score, line_color="black")

# boxes
p.vbar(cats, 0.7, q2.score, q3.score, fill_color="#E08E79", line_color="black")
p.vbar(cats, 0.7, q1.score, q2.score, fill_color="#3B8686", line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(cats, lower.score, 0.2, 0.01, line_color="black")
p.rect(cats, upper.score, 0.2, 0.01, line_color="black")

# outliers
if not out.empty:
    p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "white"
p.grid.grid_line_width = 2
p.xaxis.major_label_text_font_size="16px"

show(p)